# Gabriel Mantilla Saltos, final projects of Data Science Course in Python

### Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

### Install complements like beatifulsoup4 and import neccesary libraries

In [3]:
#pip install beautifulsoup4
#pip install lxml
#pip install html5lib

import numpy as np 
import pandas as pd
from bs4 import BeautifulSoup
import requests
website_url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')

### Extract the column names

In [4]:
table = soup.find('table',{'class':'wikitable sortable'})
column_names = [item.get_text().rstrip() for item in table.find_all('th')]

### Extract values in rows 

In [5]:
data = []
table_body = table.find('tbody')

rows = table_body.find_all('tr')

for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])

### Put all the data into a dataframe

In [6]:
df = pd.DataFrame(data, columns=column_names)
df = df.replace(to_replace='None', value=np.nan).dropna()

### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.


In [7]:
df = df[df['Borough'] != 'Not assigned'] 
df.set_index(['Postcode','Borough'], inplace=True)

### Combined into one row with the neighborhoods separated with a comma

In [8]:
df = df.groupby(['Postcode','Borough'], sort=False).agg( ','.join)
df = df.reset_index(level=[0,1])
#In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
df.head()
print(df.shape)

(103, 3)


### Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

### I tried whi this code, but it was giving bad coordinates, so I used the csv that the tool gives me, to continue with the exercise

In [1]:
# df=df.assign(latitud="",longitud="")
# postal_code=df['Postcode']
# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="foursquare_agent")
# for i in range(df.shape[0]):
#        if  geolocator.geocode(postal_code[i]) is not None:
#            location = geolocator.geocode(postal_code[i])
#            latitud = location.latitude
#            longitud=location.longitude
#            df.loc[df.index[i],'longitud']=longitud
#            df.loc[df.index[i],'latitud']=latitud

In [9]:
# 2.- Use the Geocoder package or the csv file to create the following dataframe:     
path='https://cocl.us/Geospatial_data'
df = pd.merge(df,pd.read_csv(path),left_on='Postcode',right_on='Postal Code',how='left').drop(columns=['Postal Code'])
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
Postcode         103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
Latitude         103 non-null float64
Longitude        103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB
